# Capstone Project for Coursera IBM Data Science Professional Certificate

### import the libraries that are needed for the assignment

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Scrape the wikipedia webpage and parse the data using beautifulsoup. Records where borough was not assigned, were ommitted from the final dataframe.
### For records where neighborhood as not assigned, the borough name was assigned to the neighhorhood name per assignment instructions

In [21]:
#get data from wikipedia page
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})

# define the dataframe columns
column_names = ['Postcode','Borough', 'Neighborhood'] 

# instantiate the dataframe
Canada_neighborhoods = pd.DataFrame(columns=column_names) 

#loop through the table rows and columns to pull the data
table_body = table.find('tbody')
rows = table_body.find_all('tr')
column_count = 0
m_postcode = ""
m_borough = ""
m_neighborhood = ""

for tr in rows:
    cols = tr.find_all('td')
    for td in cols:
        if column_count == 0:
            m_postcode = td.text
            column_count = column_count + 1
        elif column_count == 1:
            m_borough = td.text
            column_count = column_count + 1
        else:
            m_neighborhood = td.text
            column_count = 0
    
       
    # replace neighborhood name with borough name if neighborhood name is not assigned
    if m_neighborhood == "Not assigned\n":
        m_neighborhood = m_borough
   
    # ignore records that have borough not assigned
    if m_postcode == "":
        pass
    else:
        if m_borough == "Not assigned":
            pass
        else:
            Canada_neighborhoods = Canada_neighborhoods.append({'Postcode': m_postcode,
                                                 'Borough': m_borough,
                                                'Neighborhood': m_neighborhood}, ignore_index=True)

#the neighborhood name came with a new line character '\n' so I had to clean it with this statement
Canada_neighborhoods['Neighborhood'] = Canada_neighborhoods['Neighborhood'].map(lambda x: x.rstrip('\n')) 
Canada_neighborhoods.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### I made the assumption that the same boroughs will have the same postalcode, so I grouped by postal code and borough, and then concatencated the neighborhood name

In [27]:
New_Canada_neighborhoods = Canada_neighborhoods.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
New_Canada_neighborhoods.head(20)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [28]:
New_Canada_neighborhoods.shape

(103, 3)